## Neural network

Here we implement a Siamese neural network to test how deep learning approach works with face recognition when having small amount of data.

In [ ]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 394.3 MB 10 kB/s 
     |████████████████████████████████| 394.3 MB 16 kB/s 
     |████████████████████████████████| 132 kB 46.8 MB/s 
     |████████████████████████████████| 3.8 MB 44.7 MB/s 
     |████████████████████████████████| 462 kB 88.0 MB/s 
     |████████████████████████████████| 2.9 MB 51.3 MB/s 
     |████████████████████████████████| 14.8 MB 45.5 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68720 sha256=4ded0a83f3693585790a36f24438ddf39f941d797248a2bb497853ded39176ea
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Atte

In [ ]:
# Import dependencies and set memory growth using gpu

import os
import cv2
import random
import numpy as np
from matplotlib import pyplot as plt
from sklearn.datasets import fetch_lfw_people
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

#Create folders for positive, negative and anchor images to train and test the network

POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

# Unpack the database
!tar -xf lfw.tgz

# Here are the names of people whose number of photos of the database is >= 60. 
# We take this number because we want to test our neural network 
# on the same people we used in the Eigenfaces algorithm. 

lst_names = ['Ariel_Sharon', 'Colin_Powell', 'Donald_Rumsfeld', \
             'George_W_Bush', 'Gerhard_Schroeder', 'Hugo_Chavez', \
             'Junichiro_Koizumi', 'Tony_Blair']


# Fill positive, negative and anchor folders with data from the LFW database
# (people who have more than 60 photos)

for directory in os.listdir('lfw'):
  if os.path.basename(directory) in lst_names:

    POS_PATH_DIR = os.path.join('data', 'positive', directory)
    os.makedirs(POS_PATH_DIR)
    lstdir = os.listdir(os.path.join('lfw', directory))
    
    for file in lstdir[:len(lstdir)//2]:
      EX_PATH = os.path.join('lfw', directory, file)      
      POS_PATH_DIR_FILE = os.path.join('data', 'positive', os.path.basename(directory), file)
      os.replace(EX_PATH, POS_PATH_DIR_FILE)

    ANC_PATH_DIR = os.path.join('data', 'anchor', os.path.basename(directory))
    os.makedirs(ANC_PATH_DIR)

    for file in lstdir[len(lstdir)//2:(len(lstdir)//2)*2]:
      EX_PATH = os.path.join('lfw', directory, file)
      ANC_PATH_DIR_FILE = os.path.join('data', 'anchor', os.path.basename(directory), file)
      os.replace(EX_PATH, ANC_PATH_DIR_FILE)


for directory in os.listdir('lfw'):
  if os.path.basename(directory) not in lst_names:
    lstdir = os.listdir(os.path.join('lfw', directory))
    EX_PATH = os.path.join('lfw', directory, lstdir[0])
    NEG_PATH_DIR_FILE = os.path.join('data', 'negative', lstdir[0])
    os.replace(EX_PATH, NEG_PATH_DIR_FILE)


tar: lfw.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


FileNotFoundError: ignored

We have decided to test the network only on one person (Ariel Sharon), as training on all the people who have >60 photos in the database would take too much time. Nevertheless, for each person there would be a separate training needed and we can make conclusions based on one example.


In [ ]:
# Getting images from directories. 

anchor = tf.data.Dataset.list_files('data/anchor/Ariel_Sharon/*.jpg').take(len(os.listdir('data/anchor/Ariel_Sharon')))
positive = tf.data.Dataset.list_files('data/positive/Ariel_Sharon/*.jpg').take(len(os.listdir('data/positive/Ariel_Sharon')))
negative = tf.data.Dataset.list_files('data/negative/*.jpg').take(len(os.listdir('data/positive/Ariel_Sharon')))

# Read and preprocess (scale and resize) image

def preprocess(file_path):
    
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img, (100,100))
    img = img / 255.0

    return img

# Make labelled dataset with triplets -> (anchor, positive/negative, 1/0)

positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

# Preprocess triplets created in the previous function

def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

# Preprocess all the data

data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=104)

# Split the data into train and test parts

train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(8)
train_data = train_data.prefetch(4)

test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(8)
test_data = test_data.prefetch(4)



In [ ]:
# Transform an image matrix into model with a vector of features 

def make_embedding(): 
    inp = Input(shape=(100,100,3), name='input_image')
    
    c1 = Conv2D(64, (10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1)
    
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)
    
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)
    
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)
    
    return Model(inputs=[inp], outputs=[d1], name='embedding')

embedding = make_embedding()
embedding.summary()

# Create class that calculates the distance between vector's points

class L1Dist(Layer):
    
    def __init__(self, **kwargs):
        super().__init__()
       
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In this block of code we build and train our Siamese neural network. As we have already run and trained it, there s no need to run this block of code. In the following chunks we use previously saved file with neural network 'siamesemodelv2.h5'. 

In [ ]:
def make_siamese_model(): 
    
    # Anchor image 
    input_image = Input(name='input_img', shape=(100,100,3))
    
    # Validation image (positive/negative)
    validation_image = Input(name='validation_img', shape=(100,100,3))
    
    # Calculate distances
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))
    
    # Classify
    classifier = Dense(1, activation='sigmoid')(distances)
    
    return Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

# Calculate possible accuracy and optimize model

binary_cross_loss = tf.losses.BinaryCrossentropy()
opt = tf.keras.optimizers.Adam(1e-4) # 0.0001 

# os.makedirs(os.path.join('training_checkpoints'))

siamese_model = make_siamese_model()
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

# Train the model

@tf.function
def train_step(batch):
    
    with tf.GradientTape() as tape:     
        X = batch[:2]
        y = batch[2]
        
        yhat = siamese_model(X, training=True)
        loss = binary_cross_loss(y, yhat)
        
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
        
    return loss

def train(data, EPOCHS):

    # Iterate over all of the data exactly once 
    
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        
        r = Recall()
        p = Precision()
        
        # Iterate over batches
        for idx, batch in enumerate(data):

            loss = train_step(batch)
            yhat = siamese_model.predict(batch[:2])
            r.update_state(batch[2], yhat)
            p.update_state(batch[2], yhat) 
            progbar.update(idx+1)
        print(loss.numpy(), r.result().numpy(), p.result().numpy())
        
        # Save checkpoints
        if epoch % 10 == 0: 
            checkpoint.save(file_prefix=checkpoint_prefix)


# We iterate over all of the data 50 times while training, 
# and each time through every batch 

EPOCHS = 50
train(train_data, EPOCHS)


In [ ]:
# Importing Precision and Recall to evaluate results

from tensorflow.keras.metrics import Precision, Recall

# Get a batch of test data

test_input, test_val, y_true = test_data.as_numpy_iterator().next()
y_hat = siamese_model.predict([test_input, test_val])

# Evaluating the results 

y_hat = [1 if prediction > 0.5 else 0 for prediction in y_hat]

m = Recall()
m.update_state(y_true, y_hat)
m.result().numpy()

m = Precision()

m.update_state(y_true, y_hat)

m.result().numpy()

r = Recall()
p = Precision()

for test_input, test_val, y_true in test_data.as_numpy_iterator():
    yhat = siamese_model.predict([test_input, test_val])
    r.update_state(y_true, yhat)
    p.update_state(y_true,yhat) 
    
print('Accuracy:')
print(r.result().numpy(), p.result().numpy())

# Make plots
plt.figure(figsize=(10,8))

plt.subplot(1,2,1)
plt.imshow(test_input[0])

plt.subplot(1,2,2)
plt.imshow(test_val[0])

plt.show()

# # Save model
# siamese_model.save('siamesemodelv2.h5')

In [ ]:
# Reload model 
siamese_model = tf.keras.models.load_model('siamesemodelv2.h5', 
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})
# Make predictions with reloaded model
siamese_model.predict([test_input, test_val])

# View model summary
siamese_model.summary()

## Conclusion

After testing our neural network several times, we can see that Siamese neural network and deep learning approach in general works not well with small amounts of data. As we have only 38 photos in positive and anchor folders, the network did not have enough data to train and work good enough and accuracy of recognition was not high. As a result, we got errors in facial recognition quite often. From this we can conclude that Eigenfaces approach works much better on small amounts of data. 